<a href="https://colab.research.google.com/github/JibrilTheUltimateFlugel/TensorflowTest/blob/main/TensorflowTutorialNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1
    
    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1
  
  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
17465344/17464789 [==============================] - 1s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [3]:
# Lets look at one review
train_data[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

In [4]:
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 69s 62ms/step - loss: 0.5262 - acc: 0.7242 - val_loss: 0.2806 - val_acc: 0.8876
Epoch 2/10
625/625 [==============================] - 38s 60ms/step - loss: 0.2457 - acc: 0.9079 - val_loss: 0.2652 - val_acc: 0.8950
Epoch 3/10
625/625 [==============================] - 37s 60ms/step - loss: 0.1799 - acc: 0.9352 - val_loss: 0.2725 - val_acc: 0.9002
Epoch 4/10
625/625 [==============================] - 37s 60ms/step - loss: 0.1480 - acc: 0.9493 - val_loss: 0.2764 - val_acc: 0.8944
Epoch 5/10
625/625 [==============================] - 37s 60ms/step - loss: 0.1261 - acc: 0.9576 - val_loss: 0.3033 - val_acc: 0.8914
Epoch 6/10
625/625 [==============================] - 37s 60ms/step - loss: 0.1089 - acc: 0.9632 - val_loss: 0.3352 - val_acc: 0.8884
Epoch 7/10
625/625 [==============================] - 38s 60ms/step - loss: 0.0892 - acc: 0.9702 - val_loss: 0.3215 - val_acc: 0.8862
Epoch 8/10
625/625 [==============================] - 37s 59ms

In [8]:
results = model.evaluate(test_data, test_labels)
print(results)

782/782 [==============================] - 13s 16ms/step - loss: 0.4297 - acc: 0.8568
[0.42969444394111633, 0.8568000197410583]


In [9]:
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)


1646592/1641221 [==============================] - 0s 0us/step
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 

In [10]:
# while were at it lets make a decode function

reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
    PAD = 0
    text = ""
    for num in integers:
      if num != PAD:
        text += reverse_word_index[num] + " "

    return text[:-1]
  
print(decode_integers(encoded))

that movie was just amazing so amazing


In [11]:
# now time to make a prediction

def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred) 
  print(result[0])

positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
predict(negative_review)

[0.89587575]
[0.28983203]


In [12]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [13]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [14]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [15]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [16]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [17]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [18]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [19]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [20]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [21]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [22]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [23]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [24]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_1 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [25]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [26]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 1.66827044e-03  1.55216677e-03 -2.17856327e-03 ... -3.14404350e-03
    2.18022708e-03 -2.39616693e-05]
  [ 3.51096457e-03 -1.64700861e-04  5.73062105e-03 ...  5.24737479e-05
    6.81704050e-03  2.17994954e-03]
  [ 5.88883739e-03  1.64491881e-03  8.02147295e-03 ...  6.90988614e-04
    7.34932907e-03  2.50946637e-03]
  ...
  [ 8.60568788e-03 -3.89562361e-03  1.34243546e-02 ... -3.23499134e-03
   -4.20431979e-03  8.39823019e-03]
  [ 8.14974122e-03 -1.25051849e-03  7.25988485e-03 ... -4.41040518e-03
   -3.64979776e-03  5.95726585e-03]
  [ 9.28679015e-03  2.16523325e-03  8.27038754e-03 ... -1.49674551e-03
   -2.25334009e-03  5.07457927e-03]]

 [[ 3.75708169e-03 -1.75408169e-03  2.52010883e-03 ... -5.59973065e-03
    3.44440527e-03  1.85888272e-03]
  [ 5.15085598e-03 -4.14640550e-03  4.95234411e-03 ... -5.15715079e-03
   -3.90450004e-03  3.99021385e-03]
  [ 5.51201124e-03 -1.11230195e-03  3.47935042e-04 ... -6.21052226e-03
   -2.42231530e-03  2.97582266e-03]
  ...
  [-1.868

In [27]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 1.6682704e-03  1.5521668e-03 -2.1785633e-03 ... -3.1440435e-03
   2.1802271e-03 -2.3961669e-05]
 [ 3.5109646e-03 -1.6470086e-04  5.7306211e-03 ...  5.2473748e-05
   6.8170405e-03  2.1799495e-03]
 [ 5.8888374e-03  1.6449188e-03  8.0214730e-03 ...  6.9098861e-04
   7.3493291e-03  2.5094664e-03]
 ...
 [ 8.6056879e-03 -3.8956236e-03  1.3424355e-02 ... -3.2349913e-03
  -4.2043198e-03  8.3982302e-03]
 [ 8.1497412e-03 -1.2505185e-03  7.2598848e-03 ... -4.4104052e-03
  -3.6497978e-03  5.9572659e-03]
 [ 9.2867902e-03  2.1652333e-03  8.2703875e-03 ... -1.4967455e-03
  -2.2533401e-03  5.0745793e-03]], shape=(100, 65), dtype=float32)


In [28]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 1.6682704e-03  1.5521668e-03 -2.1785633e-03 -2.3368769e-03
  3.5667752e-03 -3.3412850e-04  8.0102799e-04  2.0578711e-03
  4.1105431e-03  4.6432470e-03 -8.2232308e-04  2.0946393e-04
  2.2583038e-03  3.9400649e-03  3.5549940e-03  2.8132533e-03
 -1.7047818e-04  3.1942835e-03  4.9534659e-03 -5.4903463e-03
 -3.4515403e-04 -1.5631063e-03  3.5062726e-03  4.3356279e-03
 -1.5077464e-03  1.2654602e-03 -9.7390637e-04 -2.1757800e-03
  3.1903572e-03 -2.4386647e-03 -3.3281003e-03 -1.9536132e-03
  1.3816865e-03 -1.7728631e-03  3.8139641e-04  6.0047442e-03
  5.4533640e-03  4.4890816e-04  5.9374310e-03 -2.6946471e-03
  2.4641142e-03  1.8618285e-03  5.0086423e-04 -4.5922599e-03
  1.7566068e-04 -2.6001660e-03  2.3825706e-03  5.6884559e-03
 -2.9879131e-03 -2.5320104e-03  7.1012502e-04  1.5043099e-03
 -5.9142513e-03 -1.7696500e-03  1.7350102e-03  2.2123619e-03
 -2.4750601e-03  4.2750239e-03 -3.8939866e-03 -3.0016684e-04
  4.5350485e-04 -4.8900056e-03 -3.1440435e-03  2.1802271e-03
 -2.396166

In [29]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"A$$n.Ynttu\njmPTlrdO\nAovqK\nZXTnnZYLUM ZO\nG-BxejAedYNaaCS;W3A$yqjvdhF3Nf:.'AMLZNWO$Qx.3Fhj!ftoWWUf3DHt"

In [30]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [31]:
model.compile(optimizer='adam', loss=loss)

In [32]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [33]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 31s 160ms/step - loss: 3.0838
Epoch 2/50
172/172 [==============================] - 29s 161ms/step - loss: 1.9969
Epoch 3/50
172/172 [==============================] - 29s 162ms/step - loss: 1.7046
Epoch 4/50
172/172 [==============================] - 29s 162ms/step - loss: 1.5457
Epoch 5/50
172/172 [==============================] - 29s 163ms/step - loss: 1.4498
Epoch 6/50
172/172 [==============================] - 29s 163ms/step - loss: 1.3825
Epoch 7/50
172/172 [==============================] - 29s 162ms/step - loss: 1.3329
Epoch 8/50
172/172 [==============================] - 29s 162ms/step - loss: 1.2958
Epoch 9/50
172/172 [==============================] - 29s 162ms/step - loss: 1.2600
Epoch 10/50
172/172 [==============================] - 29s 162ms/step - loss: 1.2203
Epoch 11/50
172/172 [==============================] - 29s 162ms/step - loss: 1.1861
Epoch 12/50
172/172 [==============================] - 29s 162ms/step - lo

In [34]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [40]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [39]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

AttributeError: ignored

In [37]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [44]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: capulet
capulets forfeir of the side
Silver a word, which at home is mackion.

ESCALUS:
Why, that my daughter? where is my wound to hear?

KATHARINA:
Milloppiness attended home,
And I am ready stary riches flown;
As to seem the bottom of my grief,
Toou plot
the insinuating nod and Citizens:
All things that from his speed
Through lamentably commended
him; his unstuff'd out a carfful page?
Besides, I have pardon
With cozen wips of my certainly acrown,
Stands keep the maid: and, like a undertaking.

Third Citizen:
But there was son.

LADY ANNE:
What down this was thick, that holp to joy,
And they high fortune eng;
The correct be the worst for brand: say that Friar John Bolingbroke.

KING RICHARD II:
Why, that's the friar of boldness live, upon
He will ask thou had run swift, he should be er,
I pritten with 
